# Spiral classification

Given the x-y coordinates of 2D points, classify them into one of three possible spiral branches that they belong to.

Objectives:
 - Build and train a PyTorch MLP model from scratch to classify the points to a high accuracy
 - Visualize how the points are transformed akin to: https://youtu.be/EyKiYVwrdjE?si=JiAKShLumRxbFWXA&t=1002

In [ ]:
import torch
from torch import nn, optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torch.functional as F

In [ ]:
import fastbook
from fastbook import *

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

## Create data

In [ ]:
seed = 12345
torch.manual_seed(seed)
N = 1000  # num_samples_per_class
K = 3     # num_classes

In [ ]:
# Generate spirals

t = torch.linspace(0, 1, N)
a = 0.8 * t + 0.2  # amplitude 0.2 → 1.0
X = list()
y = list()
for k in range(K):
    θ = (2 * t + k) * 2 * torch.pi / K + 0.2 * torch.randn(N)
    X.append(torch.stack((a * θ.sin(), a * θ.cos()), dim=1))
    y.append(torch.zeros(N, dtype=torch.long).fill_(k))
X = torch.cat(X)
y = torch.cat(y)

In [ ]:
# X.shape, y.shape

In [ ]:
# X[:5]

In [ ]:
# y.unique()

In [ ]:
plt.figure(figsize=(4.5,4))
plt.scatter(X[:,0], X[:,1], c=y)
plt.show()

## Dataset and DataLoader Splits and Batches Data

In [ ]:
class ToyDataset(Dataset):
    def __init__(self, X, y, split=0.2):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        x_item = self.features[index]
        y_item = self.labels[index]
        return x_item, y_item
    
    def __len__(self):
        return self.labels.shape[0]

In [ ]:
whole_ds = ToyDataset(X=X, y=y)
# len(whole_ds)

In [ ]:
def split_dataset(ds, train_percent=0.8):
    """
    ds - ToyDataset
    """

    train_size = int(len(ds) * train_percent)
    test_size = len(ds) - train_size

    train_subset, test_subset = random_split(ds, [train_size, test_size])

    X_train = ds.features[train_subset.indices]
    y_train = ds.labels[train_subset.indices]

    X_test = ds.features[test_subset.indices]
    y_test = ds.labels[test_subset.indices]

    train_ds = ToyDataset(X_train,y_train)
    test_ds = ToyDataset(X_test,y_test)
    
    return train_ds, test_ds

In [ ]:
train_ds, test_ds = split_dataset(whole_ds)

In [ ]:
# train_ds.features.shape

In [ ]:
# type(train_ds)

In [ ]:
# test_ds.features.shape

In [ ]:
torch.manual_seed(1337)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=32,
    shuffle=True,
    drop_last=True,
    num_workers=0
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=32,
    shuffle=False,
    drop_last=True,
    num_workers=0
)

In [ ]:
# for idx, (x, y) in enumerate(train_loader):
#     print(f"Batch {idx}: ", x.shape, y.shape)
#     if idx > 0:
#         break

## Model

In [ ]:
# class MLP(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, n_layers=3):
#         super().__init__() # IMPORTANT
#         # TODO: figure out later how to create a variable nn.Sequential
#         # self.n_layers = n_layers
#         self.h = hidden_size

#         self.net = nn.Sequential(
#             nn.Linear(input_size, hidden_size),
#             nn.ReLU(),
#             nn.Linear(hidden_size, hidden_size),
#             nn.ReLU(),
#             nn.Linear(hidden_size,output_size)
#         )
    
#     def forward(self, xb):
#         score = self.net(xb)
#         return score


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=3):
        super().__init__() # IMPORTANT

        self.n_layers = n_layers
        self.h = hidden_size

        layers = [nn.Linear(input_size, hidden_size)]
        for _ in range(n_layers-2):
            layers.append(nn.ReLU())
            layers.append(nn.Linear(hidden_size, hidden_size))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size,output_size))

        self.net = nn.Sequential(*layers)
    
    def forward(self, xb):
        score = self.net(xb)
        return score


In [ ]:
# initialize weights
# model = MLP(input_size = 2, hidden_size = 16, output_size = 3)

In [ ]:
# X_batch, y_batch = next(iter(train_loader))
# probs = model(X_batch[0])
# probs

In [ ]:
# pred = torch.argmax(probs)
# pred.item()

In [ ]:
# target = y_batch[0]
# target.item()

In [ ]:
# target == pred

In [ ]:
# probs = model(X_batch)
# probs.shape

In [ ]:
# preds = torch.argmax(probs, dim=-1) # now we need to specify dim
# preds.shape

In [ ]:
# targets = y_batch
# targets.shape

In [ ]:
# ((preds-targets)==0).float()

In [ ]:
# ((preds-targets)==0).float().mean().item()

## Linear Model

In [ ]:
class SuperLinear(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__() # IMPORTANT
        self.h = hidden_size

        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.Linear(hidden_size,output_size)
        )
    
    def forward(self, xb):
        score = self.net(xb)
        return score

## Training Loop

In [ ]:
# optim = torch.optim.AdamW(model.parameters(), lr=0.01)

In [ ]:
def train_epoch(model, train_loader, optim):
    # set model to train
    model.train()

    for idx, (xb, yb) in enumerate(train_loader):
        # forward pass
        score = model(xb)
        
        # cross-entropy loss 
        loss = F.cross_entropy(score, yb)

        # accuracy
        preds = torch.argmax(score, dim=-1)
        acc = ((preds - yb) == 0).float().mean().item()

        # clear old gradient from previous backprop
        optim.zero_grad()

        # compute new gradient and backprop
        loss.backward()

        # update parameter step
        optim.step()

    # set model back to eval in case 
    # other code wants to do that by default
    model.eval()

    return model, loss.item(), acc
    

In [ ]:
# train_epoch(model=model, train_loader=train_loader, optim=optim)

## Testing Loop

In [ ]:
def test_epoch(model, test_loader):
    
    # no gradient calculations
    with torch.no_grad():
        for idx, (xb, yb) in enumerate(test_loader):
            # model output
            score = model(xb)

            # loss
            loss = F.cross_entropy(score, yb)

            # accuracy
            preds = torch.argmax(score, dim=-1)
            acc = ((preds - yb) == 0).float().mean().item()
    
    return loss.item(), acc

In [ ]:
# test_epoch(model=model, test_loader=test_loader)

## Training and Testing Loop

In [ ]:
def train_test(model, train_loader, test_loader, optim, epochs=100, verbose=False):
    for e in range(epochs):
        model, train_loss, train_acc = train_epoch(model, train_loader, optim)
        test_loss, test_acc = test_epoch(model, test_loader)

        if verbose:
            if (e % 10 == 0) or (e == epochs-1):
                print(f"[Epoch {e}] Train/Test Loss: {train_loss:.2f} / {test_loss:.2f} | Train/Test Acc: {train_acc*100:.1f}% / {test_acc*100:.1f}%")
    return model

In [ ]:
# train_test(model, train_loader, test_loader, optim, verbose=True)

## Hyperparameter tuning

In [ ]:
def hyper_model(h, n_layers, lr, epochs=100, verbose=False):
    torch.manual_seed(1337)

    model = MLP(input_size = 2, hidden_size = h, n_layers=n_layers, output_size = 3)
    optim = torch.optim.AdamW(model.parameters(), lr=lr)
    
    model = train_test(model, train_loader, test_loader, optim, epochs=epochs, verbose=verbose)
    return model

In [ ]:
# model = hyper_model(h=10, n_layers=3, lr=0.1, verbose=True)

In [ ]:
# hyper_model(h=8, n_layers=3, lr=0.5, verbose=True)

In [ ]:
# hyper_model(h=8, n_layers=3, lr=0.1, verbose=True)

In [ ]:
# hyper_model(h=16, n_layers=3, lr=0.1, verbose=True)

In [ ]:
# hyper_model(h=16, n_layers=4, lr=0.1, verbose=True)

In [ ]:
# model = hyper_model(h=32, n_layers=3, lr=0.1, verbose=True)

In [ ]:
# TODO: train linear model as well

## Visualizing Model Performance

In [ ]:
preds = torch.argmax(model(X), dim=-1)
preds.shape

In [ ]:
# show all points in side-by-side subplots: model prediction vs ground truth 
fig, ax = plt.subplots(1,2, figsize=(10,4))

ax[0].scatter(X[:,0], X[:,1], c=y)
ax[1].scatter(X[:,0], X[:,1], c=preds)

ax[0].set_title('Original Spiral Classes')
ax[1].set_title('Network Predicted Spiral Classes')
plt.show()

In [ ]:
((preds - y) == 0).float().mean().item()

In [ ]:
# TODO: show all points as red or green for incorrect/correct classification vs ground truth 

In [ ]:
# TODO: show model decision boundary

In [ ]:
# TODO: show linear model results

## Visualize 2D Embeddings

In [ ]:
class MLPemb(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=3):
        super().__init__() # IMPORTANT
        self.h = hidden_size

        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,2),
            nn.Linear(2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,2),
            nn.Linear(2,output_size)
        )
    
    def forward(self, xb):
        score = self.net(xb)
        return score


In [ ]:
torch.manual_seed(1337)

model = MLPemb(input_size = 2, hidden_size = 10, output_size = 3)
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)

model = train_test(model, train_loader, test_loader, optim, verbose=True, epochs=2_000)

In [ ]:
model

In [ ]:
# empty dictionary for storing outputs of any layer
layer_outputs = {}

def get_layer_outputs(name):
    
    def hook(model, input, output):
        layer_outputs[name] = output
    
    return hook

In [ ]:
model.net[-2]

In [ ]:
# visualize the second to last layer output in 2D

# identify the layer we want outputs from
layer1 = model.net[2] # layer after first ReLU
layer2 = model.net[-2] # second to last layer

# register forward hook
layer1.register_forward_hook(get_layer_outputs('first_emb'))
layer2.register_forward_hook(get_layer_outputs('last_emb'))

# forward pass through model
# store outputs into layer_outputs dictionary
with torch.no_grad():
    scores = model(X)

h1 = layer_outputs['first_emb']
h2 = layer_outputs['last_emb']

preds = torch.argmax(scores, dim=-1)

# show all points in side-by-side subplots: model prediction vs ground truth 
fig, ax = plt.subplots(1,3, figsize=(15,4))

ax[0].scatter(X[:,0], X[:,1], c=y)
ax[1].scatter(X[:,0], X[:,1], c=preds)
ax[2].scatter(h2[:,0], h2[:,1], c=y)

ax[0].set_title('Original Spiral Classes')
ax[1].set_title("Network's Predicted Spiral Classes")
ax[2].set_title("Network's Transformed 2D Points")

plt.show()


In [ ]:
# show all points in side-by-side subplots: model prediction vs ground truth 
fig, ax = plt.subplots(1,3, figsize=(15,4))

ax[0].scatter(X[:,0], X[:,1], c=y)
ax[1].scatter(h1[:,0], h1[:,1], c=y)
ax[2].scatter(h2[:,0], h2[:,1], c=y)

ax[0].set_title('Original Spiral Classes')
ax[1].set_title("Network's Transformed LinReLU")
ax[2].set_title("Network's Transformed LinReLUx2")

plt.show()

In [ ]:
((preds - y) == 0).float().mean().item()

## Visualize Hidden Layer Outputs

(sorta)

In [ ]:
model = hyper_model(h=10, n_layers=3, lr=0.001, epochs=2000, verbose=True)

In [ ]:
model

In [ ]:
# empty dictionary for storing outputs of any layer
layer_outputs = {}

def get_layer_outputs(name):
    
    def hook(model, input, output):
        layer_outputs[name] = output
    
    return hook

In [ ]:
# (x, y, s, r, g, b)
# (x, y, r, g, b, a)
# (x, y, r_in, r1, g1, b1, r_out, r2, g2, b2)

In [ ]:
relu1 = model.net[1] # first ReLU
relu2 = model.net[3] # second ReLU

# register forward hook
relu1.register_forward_hook(get_layer_outputs('first_relu'))
relu2.register_forward_hook(get_layer_outputs('next_relu'))

# forward pass through model
# store outputs into layer_outputs dictionary
with torch.no_grad():
    scores = model(X)


h1 = layer_outputs['first_relu']
h2 = layer_outputs['next_relu']
h2.shape

### Experiments with Single Layer Output

In [ ]:
def minmax(x, dim=None, eps=1e-8):
    if dim is None:
        return (x - x.min()) / (x.max() - x.min()) + eps
    x_min = x.min(dim=dim, keepdim=True).values
    x_max = x.max(dim=dim, keepdim=True).values
    return (x - x_min) / (x_max - x_min) + eps


In [ ]:
h1.min(), h1.max()

In [ ]:
h1_soft = torch.softmax(h1, dim=-1)
h1_soft.min(), h1_soft.max()

In [ ]:
h1_soft = torch.sigmoid(h1)
h1_soft.min(), h1_soft.max()

In [ ]:
h1_soft = minmax(h1, dim=-1, eps=1e-8)
h1_soft.min(), h1_soft.max()

In [ ]:
size_scale = 100

In [ ]:
# h1_soft[:, 2].shape

In [ ]:
# torch.stack((h1_soft[:, 2],h1_soft[:, 3]), dim=1).shape

In [ ]:
features = ['x', 'y', 'C_in_R', 'C_in_G', 'C_in_B', 's_in', 'C_out_R', 'C_out_G', 'C_out_B', 's_out']
idx = torch.randperm(10)[:10]
feature_map = [features[i] for i in list(idx)]

# Split into separate arrays
x = h1_soft[:, idx[0]]
y = h1_soft[:, idx[1]]

colors_in_r = h1_soft[:, idx[2]]  
colors_in_g = h1_soft[:, idx[3]]  
colors_in_b = h1_soft[:, idx[4]]  
colors_in = torch.stack((colors_in_r,colors_in_g,colors_in_b), dim=1)

sizes_in = h1_soft[:, idx[5]] * size_scale

colors_out_r = h1_soft[:, idx[6]]  
colors_out_g = h1_soft[:, idx[7]]  
colors_out_b = h1_soft[:, idx[8]]  
colors_out = torch.stack((colors_out_r,colors_out_g,colors_out_b), dim=1)

sizes_out = h1_soft[:, idx[9]] * size_scale

idx, feature_map

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(
    x,
    y,
    s=sizes_out,
    c=colors_out
)
ax.scatter(
    x,
    y,
    s=sizes_in,
    c=colors_in
)


# ax.set_aspect('equal')
plt.show()

### Visualize Both Layer Outputs

In [ ]:
size_scale = 100_000

In [ ]:
def hidden_to_dot_features(h_out, norm=None, idx=None):
    
    def minmax(x, dim=None, eps=1e-8):
        if dim is None:
            return (x - x.min()) / (x.max() - x.min()) + eps
        x_min = x.min(dim=dim, keepdim=True).values
        x_max = x.max(dim=dim, keepdim=True).values
        return (x - x_min) / (x_max - x_min) + eps
    
    # decide on normalization scheme
    if norm == None:
        h_out = h_out
    elif norm == 'softmax':
        h_out = torch.softmax(h_out, dim=-1)
    elif norm == 'sigmoid':
        h_out = torch.sigmoid(h_out)
    elif norm == 'minmax':
        h_out = minmax(h_out, dim=-1)
    else:
        return ValueError("Invalid norm scheme, choose from ['softmax', 'sigmoid', 'minmax']")

    features = ['x', 'y', 'C_in_R', 'C_in_G', 'C_in_B', 's_in', 'C_out_R', 'C_out_G', 'C_out_B', 's_out']
    
    if idx == None:
        idx = torch.randperm(len(features))[:len(features)]
    
    feature_map = [features[i] for i in list(idx)]
    print(idx, feature_map)

    # Split into separate arrays
    x = h_out[:, idx[0]]
    y = h_out[:, idx[1]]

    colors_in_r = h_out[:, idx[2]]  
    colors_in_g = h_out[:, idx[3]]  
    colors_in_b = h_out[:, idx[4]]  
    colors_in = torch.stack((colors_in_r,colors_in_g,colors_in_b), dim=1)
    
    sizes_in = h_out[:, idx[5]] * size_scale


    colors_out_r = h_out[:, idx[6]]  
    colors_out_g = h_out[:, idx[7]]  
    colors_out_b = h_out[:, idx[8]]  
    colors_out = torch.stack((colors_out_r,colors_out_g,colors_out_b), dim=1)
    
    sizes_out = h_out[:, idx[9]] * size_scale

    if norm == None:
        colors_in = minmax(colors_in, dim=-1)
        colors_out = minmax(colors_out, dim=-1)
        sizes_in = minmax(sizes_in, dim=-1)
        sizes_out = minmax(sizes_out, dim=-1)

    return (x, y, colors_in, sizes_in, colors_out, sizes_out), idx

In [ ]:
def plot_hidden(dot_features):
    x, y, colors_in, sizes_in, colors_out, sizes_out = dot_features

    plt.figure(figsize=(6, 6))
    plt.scatter(
        x,
        y,
        s=sizes_out,
        c=colors_out
    )
    plt.scatter(
        x,
        y,
        s=sizes_in,
        c=colors_in
    )

    plt.show()


In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h1)
plot_hidden(dot_features)
dot_features, idx = hidden_to_dot_features(h_out=h2, idx=idx)
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h1, idx=[2, 8, 0, 1, 3, 4, 5, 6, 7, 9])
plot_hidden(dot_features)
dot_features, idx = hidden_to_dot_features(h_out=h2, idx=idx)
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h1, norm='minmax', idx=[3, 6, 7, 9, 1, 5, 2, 4, 8, 0])
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h2, norm='minmax', idx=[3, 6, 7, 9, 1, 5, 2, 4, 8, 0])
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h1, norm='sigmoid', idx=[3, 6, 7, 9, 1, 5, 2, 4, 8, 0])
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h2, norm='sigmoid', idx=[3, 6, 7, 9, 1, 5, 2, 4, 8, 0])
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h1, norm='minmax')
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h2, norm='minmax',idx=idx)
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h1, norm='sigmoid')
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h2, norm='sigmoid',idx=idx)
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h1, norm='minmax',idx=idx)
plot_hidden(dot_features)

In [ ]:
dot_features, idx = hidden_to_dot_features(h_out=h2, norm='minmax',idx=idx)
plot_hidden(dot_features)

### Is `MLP` the same as `MLPuw` model?

In [ ]:
class MLPuw(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__() # IMPORTANT

        self.h = hidden_size

        self.linear1 = nn.Linear(input_size, hidden_size)
        self.nonlin1 = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.nonlin2 = nn.ReLU()
        self.linear3 = nn.Linear(hidden_size, output_size)

    
    def forward(self, xb):
        xb = self.linear1(xb)
        xb = self.nonlin1(xb)
        xb = self.linear2(xb)
        xb = self.nonlin2(xb)
        xb = self.linear3(xb)
        output = xb.sigmoid()
        return output


In [ ]:
torch.manual_seed(1337)

model = MLP(input_size = 2, hidden_size = 6, output_size = 3)
torch.manual_seed(1337)
optim = torch.optim.AdamW(model.parameters(), lr=0.1)
torch.manual_seed(1337)
model = train_test(model, train_loader, test_loader, optim, verbose=True)

In [ ]:
model

In [ ]:
torch.manual_seed(1337)

model = MLPuw(input_size = 2, hidden_size = 6, output_size = 3)
torch.manual_seed(1337)
optim = torch.optim.AdamW(model.parameters(), lr=0.1)
torch.manual_seed(1337)

model = train_test(model, train_loader, test_loader, optim, verbose=True)

In [ ]:
model